In [1]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import lightgbm as lgb
print(lgb.__version__)

4.3.0


In [3]:
import pandas as pd

In [4]:
dataset=pd.read_csv("insurance_pre.csv")

In [5]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [7]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,True,False
1334,18,31.920,0,2205.98080,False,False
1335,18,36.850,0,1629.83350,False,False
1336,21,25.800,0,2007.94500,False,False


In [8]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [9]:
independent=dataset.iloc[:, [0, 1, 2, 4, 5]]  #independent would contain a DataFrame with all rows, 
#but only the columns specified by their integer positions: the first, second, third, fifth, and sixth columns of dataset.

In [10]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,False,True
1,18,33.770,1,True,False
2,28,33.000,3,True,False
3,33,22.705,0,True,False
4,32,28.880,0,True,False
...,...,...,...,...,...
1333,50,30.970,3,True,False
1334,18,31.920,0,False,False
1335,18,36.850,0,False,False
1336,21,25.800,0,False,False


In [11]:
dependent=dataset[['charges']]

In [12]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
from lightgbm import LGBMRegressor

In [15]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)

Number of CPU cores: 8


In [41]:
param_grid = {
    'num_leaves': [40],
    'min_child_samples': [10, 20, 30],
    'max_depth': [3, 5, 7],
    'n_estimators': [10,100,200]
}

In [42]:
grid = GridSearchCV(LGBMRegressor(), param_grid, cv=3, verbose=3, n_jobs=-1)
#grid = GridSearchCV(estimator=LGBMRegressor(), param_grid=param_grid, cv=3, verbose=3, n_jobs=-1)
grid.fit(independent,dependent)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 1338, number of used features: 5
[LightGBM] [Info] Start training from score 13270.422260
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

GridSearchCV(cv=3, estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7],
                         'min_child_samples': [10, 20, 30],
                         'n_estimators': [10, 100, 200], 'num_leaves': [40]},
             verbose=3)

In [43]:
print(independent.columns)

Index(['age', 'bmi', 'children', 'sex_male', 'smoker_yes'], dtype='object')


In [44]:
re=grid.cv_results_
print(re)
#Printing grid.cv_results_ will provide detailed information about the cross-validation results 
#for each combination of hyperparameters

{'mean_fit_time': array([0.21103581, 0.26537093, 0.45038867, 0.01392968, 0.10546366,
       0.20832666, 0.0248425 , 0.1018463 , 0.19599279, 0.05810173,
       0.27970505, 0.40895065, 0.02405477, 0.20496329, 0.25469526,
       0.02186306, 0.1258866 , 0.22271244, 0.04030156, 0.27357244,
       0.59760888, 0.03953497, 0.29715856, 0.5035708 , 0.02929036,
       0.22546458, 0.2976555 ]), 'std_fit_time': array([0.0899502 , 0.00740864, 0.12331203, 0.00778014, 0.00552487,
       0.00770625, 0.01129875, 0.01611769, 0.00185827, 0.00448177,
       0.02264848, 0.00978955, 0.00136204, 0.0268452 , 0.010497  ,
       0.00182399, 0.014495  , 0.00716545, 0.00148199, 0.01457392,
       0.02482899, 0.00125884, 0.02000096, 0.05831462, 0.0049688 ,
       0.01450278, 0.02694918]), 'mean_score_time': array([0.00133753, 0.00703001, 0.0035305 , 0.00575638, 0.00428971,
       0.00425148, 0.00376789, 0.00372124, 0.00612505, 0.00347582,
       0.00499582, 0.00313218, 0.00341201, 0.0043819 , 0.00644263,
       0.0

In [45]:
print("The R2_score value for the best parameter{}:".format(grid.best_params_))

The R2_score value for the best parameter{'max_depth': 3, 'min_child_samples': 30, 'n_estimators': 100, 'num_leaves': 40}:


In [46]:
print("Best R^2 score:", grid.best_score_)

Best R^2 score: 0.8551299491457899


In [47]:
table=pd.DataFrame.from_dict(re)

In [48]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_child_samples,param_n_estimators,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.211036,0.089950,0.001338,0.000946,3,10,10,40,"{'max_depth': 3, 'min_child_samples': 10, 'n_e...",0.770625,0.727944,0.734935,0.744501,0.018691,25
1,0.265371,0.007409,0.007030,0.006777,3,10,100,40,"{'max_depth': 3, 'min_child_samples': 10, 'n_e...",0.866135,0.856263,0.835863,0.852754,0.012605,3
2,0.450389,0.123312,0.003531,0.000211,3,10,200,40,"{'max_depth': 3, 'min_child_samples': 10, 'n_e...",0.860026,0.846057,0.831206,0.845763,0.011768,7
3,0.013930,0.007780,0.005756,0.006034,3,20,10,40,"{'max_depth': 3, 'min_child_samples': 20, 'n_e...",0.770734,0.727660,0.736007,0.744801,0.018652,24
4,0.105464,0.005525,0.004290,0.001092,3,20,100,40,"{'max_depth': 3, 'min_child_samples': 20, 'n_e...",0.868231,0.856467,0.840290,0.854996,0.011454,2
5,0.208327,0.007706,0.004251,0.000668,3,20,200,40,"{'max_depth': 3, 'min_child_samples': 20, 'n_e...",0.863297,0.848348,0.835665,0.849103,0.011294,5
6,0.024843,0.011299,0.003768,0.000219,3,30,10,40,"{'max_depth': 3, 'min_child_samples': 30, 'n_e...",0.770035,0.727703,0.740063,0.745933,0.017774,23
7,0.101846,0.016118,0.003721,0.000821,3,30,100,40,"{'max_depth': 3, 'min_child_samples': 30, 'n_e...",0.867991,0.858106,0.839293,0.855130,0.011904,1
8,0.195993,0.001858,0.006125,0.001995,3,30,200,40,"{'max_depth': 3, 'min_child_samples': 30, 'n_e...",0.864522,0.849329,0.834629,0.849493,0.012204,4
9,0.058102,0.004482,0.003476,0.000250,5,10,10,40,"{'max_depth': 5, 'min_child_samples': 10, 'n_e...",0.777601,0.733675,0.735708,0.748994,0.020245,19


In [53]:
import pickle
import lightgbm

# Trained model object (assuming it's already trained and stored in the 'regressor' variable)
regressor = lightgbm.LGBMRegressor(n_estimators=100)
regressor.fit(independent,dependent)

# Save the trained model to a file
filename = "finalized_model_LGBM.sav"
pickle.dump(regressor, open(filename, 'wb'))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000091 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 1338, number of used features: 5
[LightGBM] [Info] Start training from score 13270.422260


In [54]:
import pickle
import pandas as pd

# Load the trained model
loaded_model = pickle.load(open("finalized_model_LGBM.sav", 'rb'))

# Define the prediction input
prediction = [[30, 22, 1, 1, 0]]
prediction_df = pd.DataFrame(prediction, columns=['age', 'bmi', 'children', 'sex_male', 'smoker_yes'])

# Make the prediction
result = loaded_model.predict(prediction_df)
print(result)

[5215.33319702]
